# Tutorial - Using Appen Annotated Data with NVIDIA Transfer Learning Toolkit

The annnotated data generated with ADAP can also be used with NVIDIA's Transfer Learning Toolkit (TLT). TLT is a python based AI toolkit for customizing pre-trained AI models with your own data. TLT allows you to train, fine tune, prune and export highly optimized and accurate AI models for edge deployment by adapting popular network architectures and backbones to your data.

Our [previous tutorial](./Tutorial%20-%20Annotate%20Objects%20In%20An%20Image%20Using%20A%20Bounding%20Box%20with%20the%20Appen%20Data%20Annotation%20Platform.ipynb) explains how to generate annotations using ADAP in the Pascal VOC format for the coco-animals dataset. In this tutorial, we use that data to train an object detection model using NVIDIA's TLT.

The Transfer Learning Toolkit provides various object detection models to fine tune from:
- [Transfer Learning with Object Detection](https://ngc.nvidia.com/catalog/models/nvidia:tlt_pretrained_object_detection) with common architectures like YOLOV3, FasterRCNN, SSD, DSSD, and RetinaNet. 
- [Transfer Learning with Detectnet v2](https://ngc.nvidia.com/catalog/models/nvidia:tlt_pretrained_detectnet_v2) 

For simplicity, we create a YOLOV3 model using NVIDIA's instructions [here](https://developer.nvidia.com/blog/preparing-state-of-the-art-models-for-classification-and-object-detection-with-tlt/)

In [29]:
import os
import glob
from xml_to_kitti import xml_to_kitti

## Convert Annotations from Pascal VOC to Kitti format
We use the [xml_to_kitti.py](./xml_to_kitti.py) helper script to convert our annotations into KITTI format

In [23]:
# The classes in our coco-animals example
classes = ['bear', "bird", "cat", "dog", "giraffe", "horse", "sheep", "zebra"]

# The path to the dataset and Pascal annotations
pascal_annotations_folder = "/Users/rparundekar/Downloads/coco-animals/annotations"
pascal_train_annotations_folder = os.path.join(pascal_annotations_folder, "train")
pascal_val_annotations_folder = os.path.join(pascal_annotations_folder, "val")

In [24]:
# Create output dirs for kitti format
kitti_annotations_folder = "/Users/rparundekar/Downloads/coco-animals/kitti_annotations"
kitti_train_annotations_folder = os.path.join(kitti_annotations_folder, "train")
kitti_val_annotations_folder = os.path.join(kitti_annotations_folder, "val")

# Create the folder if it doesn't exist
if not os.path.exists(kitti_annotations_folder):
    os.makedirs(kitti_annotations_folder)
# Also create train and val folders
if not os.path.exists(kitti_train_annotations_folder):
    os.makedirs(kitti_train_annotations_folder)
if not os.path.exists(kitti_val_annotations_folder):
    os.makedirs(kitti_val_annotations_folder)

In [25]:
# Convert train and val data to kitti
xml_to_kitti(input_dir=pascal_train_annotations_folder, output_dir=kitti_train_annotations_folder, encode_difficult="0", classes=classes)

In [31]:
# Print one file content from the training set
for filename in glob.glob(os.path.join(kitti_annotations_folder, 'train', "*.txt")):
    print(f"File: {filename}")
    print(f"Contents:")
    with open(os.path.join(kitti_annotations_folder, 'train', filename), "r") as f:
        print(f.read())
    break

File: /Users/rparundekar/Downloads/coco-animals/kitti_annotations/train/COCO_train2014_000000564611.txt
Contents:
cat 0 0 0 165 173 214 208 0 0 0 0 0 0 0
cat 0 0 0 165 173 214 208 0 0 0 0 0 0 0
cat 0 0 0 265 190 267 192 0 0 0 0 0 0 0
cat 0 0 0 265 190 267 192 0 0 0 0 0 0 0



## Training with TLT 

In [32]:
#Install tlt-launcher
!pip3 install nvidia-pyindex
!pip3 install nvidia-tlt

    ERROR: Command errored out with exit status 1:
     command: /usr/local/opt/python@3.9/bin/python3.9 -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/vl/7s4plzcx3yxcvpkf5n5zg2pm0000w4/T/pip-install-av1tzrhb/nvidia-pyindex_79c54bb1c433442ba417658b81e0ffe8/setup.py'"'"'; __file__='"'"'/private/var/folders/vl/7s4plzcx3yxcvpkf5n5zg2pm0000w4/T/pip-install-av1tzrhb/nvidia-pyindex_79c54bb1c433442ba417658b81e0ffe8/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/vl/7s4plzcx3yxcvpkf5n5zg2pm0000w4/T/pip-pip-egg-info-7onj9k87
         cwd: /private/var/folders/vl/7s4plzcx3yxcvpkf5n5zg2pm0000w4/T/pip-install-av1tzrhb/nvidia-pyindex_79c54bb1c433442ba417658b81e0ffe8/
    Complete output (6 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/private/var/